In [1]:
import magma as m
from mantle import FullAdder

## class Add2 - Defining a Circuit

Now let's build a 2-bit adder using `FullAdder`. 
We'll use a simple ripple carry adder design by connecting the carry out of one full adder
to the carry in of the next full adder.
The resulting adder will accept as input a carry in,
and generate a final carry out. Here's a `logisim` diagram of the circuit we will construct:

![2-Bit Adder](logisim/adder.png)

In [2]:
class Add2(m.Circuit):
    io = m.IO(I0=m.In(m.UInt[2]), I1=m.In(m.UInt[2]), CIN=m.In(m.Bit),
              O=m.Out(m.UInt[2]), COUT=m.Out(m.Bit))
    n = len(io.I0)
    O = []
    COUT = io.CIN
    for i in range(n):
        fulladder = FullAdder()
        Oi, COUT = fulladder(io.I0[i], io.I1[i], COUT)
        O.append(Oi)
    io.O <= m.uint(O)
    io.COUT <= COUT

Although we are making an 2-bit adder,
we do this using a for loop that can be generalized to construct an n-bit adder.
Each time through the for loop we create an instance of a full adder 
by calling `FullAdder()`. 
Recall that circuits are python classes,
so that calling a class returns an instance of that class.

Note how we wire up the full adders.
Calling an circuit instance has the effect of wiring
up the arguments to the inputs of the circuit.
That is,
```
O, COUT = fulladder(I0, I1, CIN)
```
is equivalent to
```
m.wire(IO, fulladder.I0)
m.wire(I1, fulladder.I1)
m.wire(CIN, fulladder.CIN)
O = fulladder.O
COUT = fulladder.COUT
```
The outputs of the circuit are returned.

Inside this loop we append single bit outputs from the full adders
to the Python list `O`. 
We also set the `CIN` of the next full adder to the `COUT` of the previous instance.

Finally, we then convert the list `O` to a `Uint(n)`. 
In addition to `Bits(n)`,
`Magma` also has built in types `UInt(n)` and `SInt(n)` 
to represent unsigned and signed ints.
`Magma` also has type conversion functions `bits`, `uint`, and `sint` to convert
between different types. 
In this example, `m.uint(C)` converts the list of bits to a `UInt(len(C))`.

## DefineAdd Generator

One question you may be asking yourself, is how can this code be generalized to produce an n-bit adder. We do this by creating an add *generator*.
A generator is a Python function that takes parameters and returns a circuit class.
Calling the generator with different parameter values will create different circuits.
The power of `Magma` results from being to use all the features of Python
to create powerful hardware generators.

Here is the code:

In [3]:
def DefineAdd(n):
    class _Add(m.Circuit):
        name = f'Add{n}'
        io = m.IO(I0=m.In(m.UInt[2]), I1=m.In(m.UInt[2]), CIN=m.In(m.Bit),
                  O=m.Out(m.UInt[2]), COUT=m.Out(m.Bit))
        O = []
        COUT = io.CIN
        for i in range(n):
            fulladder = FullAdder()
            Oi, COUT = fulladder(io.I0[i], io.I1[i], COUT)
            O.append(Oi)
        io.O <= m.uint(O)
        io.COUT <= COUT
    return _Add

def Add(n):
    return DefineAdd(n)()

def add(i0, i1, cin):
    assert len(i0) == len(i1)
    return Add(len(i0))(i0, i1, cin)

In [4]:
from magma.simulator import PythonSimulator

Add2 = DefineAdd(2)
add2 = PythonSimulator(Add2)

print(add2(1,2,0))
assert add2(1, 2,0) == (3, 0), "Failed"
print("Success!")

(3, 0)
Success!


In [5]:
m.compile("build/Add2", Add2, output="coreir-verilog")
%cat build/Add2.v

module corebit_xor (
    input in0,
    input in1,
    output out
);
  assign out = in0 ^ in1;
endmodule

module corebit_or (
    input in0,
    input in1,
    output out
);
  assign out = in0 | in1;
endmodule

module corebit_and (
    input in0,
    input in1,
    output out
);
  assign out = in0 & in1;
endmodule

module FullAdder (
    input I0,
    input I1,
    input CIN,
    output O,
    output COUT
);
wire magma_Bit_and_inst0_out;
wire magma_Bit_and_inst1_out;
wire magma_Bit_and_inst2_out;
wire magma_Bit_or_inst0_out;
wire magma_Bit_or_inst1_out;
wire magma_Bit_xor_inst0_out;
wire magma_Bit_xor_inst1_out;
corebit_and magma_Bit_and_inst0 (
    .in0(I0),
    .in1(I1),
    .out(magma_Bit_and_inst0_out)
);
corebit_and magma_Bit_and_inst1 (
    .in0(I1),
    .in1(CIN),
    .out(magma_Bit_and_inst1_out)
);
corebit_and magma_Bit_and_inst2 (
    .in0(I0),
    .in1(CIN),
    .out(magma_Bit_and_inst2_out)
);
corebit_or magma_Bit_or_inst0 (
    .in0(magma_Bit_and_inst0_out),
    .in1(magma

In [6]:
m.compile("build/Add2", FullAdder, output="coreir")
%cat build/Add2.json

{"top":"global.FullAdder",
"namespaces":{
  "global":{
    "modules":{
      "Add2":{
        "type":["Record",[
          ["I0",["Array",2,"BitIn"]],
          ["I1",["Array",2,"BitIn"]],
          ["CIN","BitIn"],
          ["O",["Array",2,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "FullAdder_inst0":{
            "modref":"global.FullAdder"
          },
          "FullAdder_inst1":{
            "modref":"global.FullAdder"
          }
        },
        "connections":[
          ["self.CIN","FullAdder_inst0.CIN"],
          ["FullAdder_inst1.CIN","FullAdder_inst0.COUT"],
          ["self.I0.0","FullAdder_inst0.I0"],
          ["self.I1.0","FullAdder_inst0.I1"],
          ["self.O.0","FullAdder_inst0.O"],
          ["self.COUT","FullAdder_inst1.COUT"],
          ["self.I0.1","FullAdder_inst1.I0"],
          ["self.I1.1","FullAdder_inst1.I1"],
          ["self.O.1","FullAdder_inst1.O"]
        ]
      },
      "FullAdder":{
        "type":["Record",[
 

In [7]:
!coreir -i build/Add2.json -p instancecount

/Users/leonardtruong/miniconda3/lib/python3.7/site-packages/pysmt/walkers/generic.py:43: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if len(nodetypes) == 1 and isinstance(nodetypes[0], collections.Iterable):
An instance count of all the primitives
FullAdder | instances in current | instances in children | 
  corebit_and | 3 | 0
  corebit_or | 2 | 0
  corebit_xor | 2 | 0

Add2 | instances in current | instances in children | 
  corebit_and | 0 | 6
  corebit_or | 0 | 4
  corebit_xor | 0 | 4

{"top":"global.FullAdder",
"namespaces":{
  "global":{
    "modules":{
      "Add2":{
        "type":["Record",[
          ["I0",["Array",2,"BitIn"]],
          ["I1",["Array",2,"BitIn"]],
          ["CIN","BitIn"],
          ["O",["Array",2,"Bit"]],
          ["COUT","Bit"]
        ]],
        "instances":{
          "FullAdder_inst0":{
            "modref":"global.FullAdder"
          },
      